# Parallel Subtask Agent Workflow

<a href="https://colab.research.google.com/github/mistralai/cookbook/blob/main/mistral/agents/non_framework/agentic_workflows/parallel_subtask_agentic_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook demonstrates how to create a generic agent workflow that automatically breaks complex tasks into multiple subtasks.

These subtasks are completed using parallel MistralAI LLM calls, enhanced with real-time information from Tavily API.

The results are then synthesized into a comprehensive response.

## Workflow Overview

1. An orchestrator LLM analyzes the main task and breaks it into distinct, parallel subtasks
2. Each subtask is assigned to a worker LLM with specialized instructions
3. Workers execute in parallel, using Tavily API for up-to-date information as needed
4. Results are synthesized into a unified response

**NOTE**: We will use MistralAI’s LLM for subtask handling and response synthesis, and the Tavily API to retrieve up-to-date real.

## Solution Architecture

![solution architecture](./images/parallel_subtask_agentic_workflow.png)

### Installation

In [1]:
!pip install -qU --upgrade pip
!pip install -qU mistralai requests

### Imports

In [2]:
import os
import json
import asyncio
import requests
from typing import Any, Optional, Dict, List, Union
from pydantic import Field, BaseModel, ValidationError
from mistralai import Mistral
from IPython.display import display, Markdown

import nest_asyncio
nest_asyncio.apply()

### Set your API keys

Here we set the API keys for `MistralAI` and `Tavily`. You can obtain the keys from the following links:

1. MistralAI: https://console.mistral.ai/api-keys
2. Tavily: https://app.tavily.com/home

In [3]:
MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY", "<YOUR MISTRAL API KEY>") # Get it from https://console.mistral.ai/api-keys
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY", "<YOUR TAVILY API KEY>") # Get it from https://app.tavily.com/home

### Initialize Mistral client

In [4]:
mistral_client = Mistral(api_key=MISTRAL_API_KEY)
MISTRAL_MODEL = "mistral-small-latest"  # Can be configured based on needs

### Tavily API configuration

In [5]:
TAVILY_API_URL = "https://api.tavily.com/search"
TAVILY_HEADERS = {
    "Authorization": f"Bearer {TAVILY_API_KEY}",
    "Content-Type": "application/json"
}

### Pydantic Models for Structured Data

Pydantic models provide data validation and serialization, ensuring the data we receive from LLMs matches our expected structure. This helps maintain consistency between the orchestrator and worker components.

**SubTask:** Individual subtask definition - defines a discrete unit of work with its type, description, and optional search query.

**TaskList:** Output structure from the orchestrator - contains analysis and a list of defined subtasks to be executed in parallel.

In [6]:
class SubTask(BaseModel):
    """Individual subtask definition"""
    task_id: str
    type: str
    description: str
    search_query: Optional[str]  # Query for Tavily search for the subtask

class TaskList(BaseModel):
    """Structure for orchestrator output"""
    analysis: str
    subtasks: List[SubTask]

### API Utility Functions

API Utility functions handle communication with external APIs and process the responses, providing clean interfaces for the rest of the workflow.

**fetch_information:** Retrieves relevant information from Tavily API based on a query and returns structured results.

**run_mistral_llm:** Executes a standard call to Mistral AI with given prompts, returning the generated content.

**parse_structured_output:** Uses Mistral's structured output capability to generate and parse responses according to Pydantic models.

In [7]:
def fetch_information(query: str, max_results: int = 3):
    """Retrieve information from Tavily API"""
    payload = {
        "query": query,
        "search_depth": "advanced",
        "include_answer": True,
        "max_results": max_results
    }

    try:
        response = requests.post(TAVILY_API_URL, json=payload, headers=TAVILY_HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Tavily: {e}")
        return {"error": str(e), "results": []}

def run_mistral_llm(prompt: str, system_prompt: Optional[str] = None):
    """Run Mistral LLM with given prompts"""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({"role": "user", "content": prompt})

    response = mistral_client.chat.complete(
        model=MISTRAL_MODEL,
        messages=messages,
        temperature=0.7,
        max_tokens=4000
    )

    return response.choices[0].message.content

def parse_structured_output(prompt: str, response_format: BaseModel, system_prompt: Optional[str] = None):
    """Get structured output from Mistral LLM based on a Pydantic model"""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({"role": "user", "content": prompt})

    response = mistral_client.chat.parse(
        model=MISTRAL_MODEL,
        messages=messages,
        response_format=response_format,
        temperature=0.2
    )

    return json.loads(response.choices[0].message.content)

### Async Worker Functions

These functions enable parallel execution of subtasks, allowing the workflow to process multiple components simultaneously for greater efficiency.

**run_task_async:** Executes a single subtask asynchronously, enhancing it with relevant information from Tavily when needed.

**execute_tasks_in_parallel:** Manages the parallel execution of all subtasks, ensuring they run concurrently and their results are properly collected.

In [8]:
async def run_task_async(task: SubTask, original_task: str):
    """Execute a single subtask asynchronously with Tavily enhancement"""

    # Prepare context with Tavily information if a search query is provided
    context = ""
    if task.search_query:
        print(f"Fetching information for: {task.search_query}")
        search_results = fetch_information(task.search_query)

        # Format search results into context
        if "results" in search_results and search_results["results"]:
            context = "### Relevant Information:\n"
            for result in search_results["results"]:
                context += f"- {result.get('content', '')}\n"

        if "answer" in search_results and search_results["answer"]:
            context += f"\n### Summary: {search_results['answer']}\n"

    # Worker prompt with task information and context
    worker_prompt = f"""
    Complete the following subtask based on the given information:

    Original Task: {original_task}

    Subtask Type: {task.type}
    Subtask Description: {task.description}

    {context}

    Please provide a detailed response for this specific subtask only.
    """

    # Use asyncio to run in a thread pool to prevent blocking
    return await asyncio.to_thread(
        run_mistral_llm,
        prompt=worker_prompt,
        system_prompt="You are a specialized agent focused on solving a specific aspect of a larger task."
    )

async def execute_tasks_in_parallel(subtasks: List[SubTask], original_task: str):
    """Execute all subtasks in parallel"""
    tasks = []
    for subtask in subtasks:
        tasks.append(run_task_async(subtask, original_task))

    return await asyncio.gather(*tasks)

## Main Workflow Function

The primary orchestration function that coordinates the entire parallel subtask process from initial request to final synthesized response.

**parallel_subtask_workflow:** Manages the complete workflow by orchestrating task decomposition, parallel execution of subtasks, and final synthesis of results into a comprehensive response.

Steps:

1. **Task Analysis:** The orchestrator analyzes the user's query and breaks it into distinct subtasks

2. **Subtask Definition:** Each subtask is defined with a unique ID, type, description, and search query

3. **Parallel Execution:** Subtasks are executed concurrently by worker agents
Information Enhancement: Workers retrieve relevant information from Tavily when needed

4. **Result Collection:** Outputs from all workers are gathered

5. **Synthesis:** Individual results are combined into a comprehensive final response

6. **Final Response:** Complete workflow results are returned, including both individual analyses and the synthesized answer

In [9]:
async def workflow(user_task: str):
    """Main workflow function to process a task through the parallel subtask agent workflow"""

    print("=== USER TASK ===\n")
    print(user_task)

    # Step 1: Orchestrator breaks down the task into subtasks
    orchestrator_prompt = f"""
    Analyze this task and break it down into 3-5 distinct, specialized subtasks that could be executed in parallel:

    Task: {user_task}

    For each subtask:
    1. Assign a unique task_id
    2. Define a specific type that describes the subtask's focus
    3. Write a clear description explaining what needs to be done
    4. Provide a search query if the subtask requires additional information

    First, provide a brief analysis of your understanding of the task.
    Then, define the subtasks that would collectively solve this problem effectively.

    Remember to make the subtasks complementary, not redundant, focusing on different aspects of the problem.
    """

    orchestrator_system_prompt = """
    You are a task orchestrator that specializes in breaking down complex problems into smaller,
    well-defined subtasks that can be solved independently and in parallel. Think carefully about
    the most logical way to decompose the given task.
    """

    print("\nOrchestrating task decomposition...")
    # Get structured output from orchestrator
    task_breakdown = parse_structured_output(
        prompt=orchestrator_prompt,
        response_format=TaskList,
        system_prompt=orchestrator_system_prompt
    )

    # Display orchestrator output
    print("\n=== ORCHESTRATOR OUTPUT ===")
    print(f"\nANALYSIS:\n{task_breakdown['analysis']}")
    print("\nSUBTASKS:")
    for task in task_breakdown["subtasks"]:
        print(f"- {task['task_id']}: {task['type']} - {task['description'][:100]}...")

    # Step 2: Execute subtasks in parallel
    print("\nExecuting subtasks in parallel...")
    subtask_results = await execute_tasks_in_parallel(
        [SubTask(**task) for task in task_breakdown["subtasks"]],
        user_task
    )

    # Display worker results
    for i, (task, result) in enumerate(zip(task_breakdown["subtasks"], subtask_results)):
        print(f"\n=== WORKER RESULT ({task['type']}) ===")
        print(f"{result[:200]}...\n")

    # Step 3: Synthesize final response
    print("\nSynthesizing final response...")

    # Format worker responses for synthesizer
    worker_responses = ""
    for task, response in zip(task_breakdown["subtasks"], subtask_results):
        worker_responses += f"\n=== SUBTASK: {task['type']} ===\n{response}\n"

    synthesizer_prompt = f"""
    Given the following task: {user_task}

    And these responses from different specialized agents focusing on different aspects of the task:

    {worker_responses}

    Please synthesize a comprehensive, coherent response that addresses the original task.
    Integrate insights from all specialized agents while avoiding redundancy.
    Ensure your response is balanced, considering all the different perspectives provided.
    """

    final_response = run_mistral_llm(
        prompt=synthesizer_prompt,
        system_prompt="You are a synthesis agent that combines specialized analyses into comprehensive responses."
    )

    return {
        "orchestrator_analysis": task_breakdown["analysis"],
        "subtasks": task_breakdown["subtasks"],
        "subtask_results": subtask_results,
        "final_response": final_response
    }

### Run workflow with an example task

Here we run the worklow with a sample example task comparing mobile phones recommendation

In [10]:
task = "Compare the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro, and recommend which one I should purchase."

In [11]:
result = asyncio.run(workflow(task))

=== USER TASK ===

Compare the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro, and recommend which one I should purchase.

Orchestrating task decomposition...

=== ORCHESTRATOR OUTPUT ===

ANALYSIS:
The task involves comparing three high-end smartphones (iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro) across various aspects such as design, performance, camera, battery life, and price. The goal is to provide a recommendation based on these comparisons. To effectively tackle this, we need to break down the task into subtasks that focus on specific aspects of the phones, gather relevant information, and synthesize the findings to make an informed recommendation.

SUBTASKS:
- compare_design: design_comparison - Compare the design, build quality, and display of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel...
- compare_performance: performance_comparison - Compare the performance, including processor, RAM, and storage options, of the iPhone 16 Pro, iPhone...
- compare_camer

### Final Response

In [12]:
print("\n=== FINAL SYNTHESIZED RESPONSE ===")

display(Markdown(result["final_response"]))


=== FINAL SYNTHESIZED RESPONSE ===


Based on the detailed analyses provided by the specialized agents, here's a comprehensive comparison of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro to help you make an informed decision:

**Design:**

* The iPhone 15 Pro stands out with its premium titanium and glass build, offering a more luxurious feel and improved durability. It also features a higher refresh rate display for smoother scrolling and better responsiveness.
* The iPhone 16 Pro has an aluminum and glass design with a smaller camera module, but it lacks the premium feel and higher refresh rate of the iPhone 15 Pro.
* The Google Pixel 9 Pro has a cleaner design with USB-C ports and side buttons, similar in size and feel to the iPhone 16 Pro but may not match the premium build quality of the iPhone 15 Pro.

**Performance:**

* The iPhone 16 Pro is expected to outperform both the iPhone 15 Pro and Google Pixel 9 Pro with its more powerful A18 Pro chip and better graphics capabilities.
* The iPhone 15 Pro, powered by the A17 Pro chip, also outperforms the Pixel 9 Pro in benchmarks.
* While the Pixel 9 Pro has more RAM (16GB compared to 8GB in iPhones), the iPhone models offer better benchmark performance and battery life.

**Camera:**

* Both the iPhone 16 Pro and Google Pixel 9 Pro offer extraordinary camera functions with fine differences. The Pixel 9 Pro stands out due to its tele-zoom, great natural blur, and night vision mode, while the iPhone 16 Pro offers superior color transitions and better night performance (excluding 8K).
* The iPhone 16 Pro has a 48MP main camera, 12MP ultra-wide, and 12MP telephoto cameras. The Google Pixel 9 Pro has a 50MP main camera, but details about its ultra-wide and telephoto cameras were not provided.
* The Pixel 9 Pro has a 42MP selfie camera (output at 10.5MP) with more detail and nicer colors compared to the iPhone's 12MP selfie camera.

**Battery Life and Charging:**

* The Google Pixel 9 Pro has the largest battery capacity (4,700 mAh) and generally offers the best battery life among the three devices. It supports fast wired and wireless charging, although its charging times are slightly slower than the iPhone 16 Pro.
* The iPhone 16 Pro has a smaller battery capacity (3,582 mAh) but still offers respectable battery life. It supports slightly faster wired and wireless charging compared to the Pixel 9 Pro and charges up more in a 30-minute period.
* The iPhone 15 Pro has the smallest battery capacity (3,290 mAh) and the shortest battery life among the three devices.

**Price and Value for Money:**

* The iPhone 16 Pro starts at $1,099, offering some improvements over the iPhone 15 Pro. However, the upgrades are minor, and the iPhone 15 Pro is still considered an incredible phone with a fantastic user experience and a powerful camera system.
* The iPhone 15 Pro starts at $999 (new) but can be found renewed on Amazon for around $600-$650 (128GB), making it an attractive option with better value for money.
* The Google Pixel 9 Pro starts at $799, offering a comparable experience to the iPhone 16 Pro in many aspects at a lower price point. The Pixel 8 Pro, which costs just $700, delivers a similar experience to the Pixel 9 Pro XL and comes with a 7-year update policy, offering even better value for money.

**Recommendation:**

* If you're looking for the latest features and don't mind paying a premium, the iPhone 16 Pro could be worth it.
* If you're looking for the best value for money, the iPhone 15 Pro (especially renewed) or the Google Pixel 9 Pro (or even the Pixel 8 Pro) would be excellent choices. The iPhone 15 Pro offers better specs compared to the iPhone 16 Pro in some aspects and can be found at a significant discount. The Google Pixel 9 Pro offers a comparable experience to the iPhone 16 Pro in many aspects at a lower price point, and the Pixel 8 Pro offers even better value for money.

Ultimately, the choice between these devices will depend on your specific needs and preferences, such as camera performance, software updates, overall user experience, and budget. Consider what features are most important to you and choose the device that best fits your requirements.

### Examining Orchestrator Analysis, Subtask information and responses

We can examine the Orchestrator Analysis, subtasks created, the corresponding search queries, and the individual responses.

In [13]:
print("\n=== Orchestrator Analysis ===\n")

display(Markdown(result['orchestrator_analysis']))


=== Orchestrator Analysis ===



The task involves comparing three high-end smartphones (iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro) across various aspects such as design, performance, camera, battery life, and price. The goal is to provide a recommendation based on these comparisons. To effectively tackle this, we need to break down the task into subtasks that focus on specific aspects of the phones, gather relevant information, and synthesize the findings to make an informed recommendation.

In [14]:
print("\n=== SUBTASKS CREATED ===\n")

for subtask in result['subtasks']:
    display(Markdown(f"- {subtask['task_id']}: \n - Task type: {subtask['type']} \n - Task Description: - {subtask['description'][:100]} \n - search_query - {subtask['search_query']}"))


=== SUBTASKS CREATED ===



- compare_design: 
 - Task type: design_comparison 
 - Task Description: - Compare the design, build quality, and display of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 
 - search_query - iPhone 16 Pro design vs iPhone 15 Pro design vs Google Pixel 9 Pro design

- compare_performance: 
 - Task type: performance_comparison 
 - Task Description: - Compare the performance, including processor, RAM, and storage options, of the iPhone 16 Pro, iPhone 
 - search_query - iPhone 16 Pro performance vs iPhone 15 Pro performance vs Google Pixel 9 Pro performance

- compare_camera: 
 - Task type: camera_comparison 
 - Task Description: - Compare the camera capabilities, including megapixels, video recording, and special features, of the 
 - search_query - iPhone 16 Pro camera vs iPhone 15 Pro camera vs Google Pixel 9 Pro camera

- compare_battery: 
 - Task type: battery_comparison 
 - Task Description: - Compare the battery life and charging capabilities of the iPhone 16 Pro, iPhone 15 Pro, and Google P 
 - search_query - iPhone 16 Pro battery vs iPhone 15 Pro battery vs Google Pixel 9 Pro battery

- compare_price: 
 - Task type: price_comparison 
 - Task Description: - Compare the pricing and value for money of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro. 
 - search_query - iPhone 16 Pro price vs iPhone 15 Pro price vs Google Pixel 9 Pro price

In [15]:
print("\n=== SUBTASKS RESULTS ===\n")

for i, subtask_result in enumerate(result['subtask_results']):

    display(Markdown(f"# Task_{i+1} Result: \n {subtask_result} \n"))


=== SUBTASKS RESULTS ===



# Task_1 Result: 
 Based on the provided information, here's a detailed design comparison of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro:

1. **Design and Build Quality:**

   - **iPhone 15 Pro:** The iPhone 15 Pro features a premium titanium and glass build, which gives it a more luxurious feel and improved durability. The use of titanium makes it lighter yet stronger than the iPhone 16 Pro. It also has thinner bezels around the display, contributing to a more modern and sleek look. The iPhone 15 Pro has buttons on both sides, which can be less convenient when using a tripod.

   - **iPhone 16 Pro:** The iPhone 16 Pro uses an aluminum and glass design, which is less premium than the titanium build of the iPhone 15 Pro. However, it has a smaller camera module, which some users might prefer for its less bulky appearance. The iPhone 16 Pro also has buttons on both sides, similar to the iPhone 15 Pro.

   - **Google Pixel 9 Pro:** The Pixel 9 Pro has a cleaner design with USB-C ports and side buttons. It lacks the extra button on the right side found on both iPhone models, which can be advantageous when using a tripod. The Pixel 9 Pro's design is similar in size and feel to the iPhone 16 Pro but may not have the same premium feel as the iPhone 15 Pro's titanium build.

2. **Display:**

   - **iPhone 15 Pro:** The iPhone 15 Pro features a display with a higher refresh rate compared to the iPhone 16 Pro, providing smoother scrolling and better overall responsiveness. This can enhance the user experience, especially when browsing the web, gaming, or using other high-performance applications.

   - **iPhone 16 Pro:** While the iPhone 16 Pro has a good display, it does not match the refresh rate of the iPhone 15 Pro. This might make the iPhone 15 Pro a better choice for users who prioritize display quality and smoothness.

   - **Google Pixel 9 Pro:** The information provided does not specify the display refresh rate of the Pixel 9 Pro. However, it is mentioned that the iPhone 15 Pro and Pixel 9 Pro have similar camera setups, which might imply that their displays are also comparable. Further information would be needed to make a definitive comparison.

In summary, the iPhone 15 Pro stands out in terms of design and build quality due to its premium titanium construction and higher refresh rate display. The iPhone 16 Pro offers a more affordable alternative with a smaller camera module but lacks the premium feel and higher refresh rate of the iPhone 15 Pro. The Google Pixel 9 Pro has a cleaner design with USB-C ports and a similar camera setup to the iPhone 16 Pro, but it may not match the premium build quality of the iPhone 15 Pro. 


# Task_2 Result: 
 Based on the provided information, here's a detailed performance comparison of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro:

1. **Processor:**
   - **iPhone 16 Pro:** Expected to feature the A18 Pro chipset, which is anticipated to be more powerful than its predecessor, the A17 Pro found in the iPhone 15 Pro.
   - **iPhone 15 Pro:** Powered by the A17 Pro chip, a 3nm processor that currently offers industry-leading performance.
   - **Google Pixel 9 Pro:** Equipped with the Tensor G4 processor, Google's latest 4nm chip.

2. **Benchmark Performance:**
   - **Geekbench 6 (Single-core):**
     - iPhone 15 Pro: 2,890
     - Pixel 9 Pro: 1,948
     - iPhone 16 Pro: Expected to score higher than the iPhone 15 Pro.
   - **Geekbench 6 (Multi-core):**
     - iPhone 15 Pro: 7,194
     - Pixel 9 Pro: 4,794
     - iPhone 16 Pro: Expected to score higher than the iPhone 15 Pro.
   - **3DMark Wild Life Unlimited (Graphics):**
     - iPhone 15 Pro: 94.4 FPS
     - Pixel 9 Pro: 55.97 FPS
     - iPhone 16 Pro: Expected to score higher than the iPhone 15 Pro.
   - **AnTuTu Benchmark:**
     - iPhone 16 Pro: 2,046K (82% better than the Pixel 9 Pro)
     - Pixel 9 Pro: 1,125K

3. **RAM and Storage:**
   - **iPhone 16 Pro:** 8GB of RAM and NVMe flash storage (no expandable storage).
   - **iPhone 15 Pro:** Likely similar to the iPhone 16 Pro, with 8GB of RAM and NVMe flash storage.
   - **Google Pixel 9 Pro:** 16GB of LPDDR5X RAM and UFS 3.1 flash storage (no expandable storage).

4. **GPU Performance:**
   - **iPhone 16 Pro:** A18 Pro chip's GPU with ~2,289 GFLOPS, 768 shading units, and a clock speed of 1,490 MHz.
   - **iPhone 15 Pro:** A17 Pro chip's GPU with slightly lower performance than the A18 Pro.
   - **Google Pixel 9 Pro:** Tensor G4 chip's GPU with ~1,684.4 GFLOPS, 896 shading units, and a clock speed of 940 MHz.

5. **Battery Life:**
   - The iPhone 16 Pro shows 28% longer battery life than the Pixel 9 Pro (28:13 hours vs. 22:07 hours).

In summary, the iPhone 16 Pro is expected to outperform both the iPhone 15 Pro and Google Pixel 9 Pro in terms of raw performance, thanks to its more powerful A18 Pro chip and better graphics capabilities. The iPhone 15 Pro also outperforms the Pixel 9 Pro in benchmarks. While the Pixel 9 Pro has more RAM, the iPhone models offer better benchmark performance and battery life. The choice between these devices will depend on your specific needs and preferences, as well as other factors like camera performance, software, and ecosystem preferences. 


# Task_3 Result: 
 Based on the provided information, here's a detailed comparison of the camera capabilities of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro:

1. **Megapixels and Sensors:**
   - **iPhone 16 Pro:**
     - Main camera: 48MP sensor with f/1.7 aperture and 24mm equivalent lens.
     - Ultra-wide camera: 12MP sensor with f/2.2 aperture and 13mm equivalent lens.
     - Telephoto camera: 12MP sensor with f/2.8 aperture and 120mm equivalent lens.
   - **iPhone 15 Pro:**
     - Main camera: 48MP sensor with f/1.7 aperture and 24mm equivalent lens (similar to iPhone 16 Pro).
     - Ultra-wide camera: 12MP sensor with f/2.2 aperture and 13mm equivalent lens (similar to iPhone 16 Pro).
     - Telephoto camera: 12MP sensor with f/2.8 aperture and 120mm equivalent lens (similar to iPhone 16 Pro).
   - **Google Pixel 9 Pro:**
     - Main camera: 50MP sensor with f/1.7 aperture and 24mm equivalent lens.
     - Ultra-wide camera: Not specified in the provided information.
     - Telephoto camera: Not specified in the provided information, but it's mentioned that the Pixel 9 Pro has a tele-zoom.

2. **Video Recording:**
   - Both the iPhone 16 Pro and Google Pixel 9 Pro offer advanced video recording modes.
   - The iPhone 16 Pro has a feature called "Cinema Mode" for creating a blurred background effect in videos.
   - The Google Pixel 9 Pro has a similar feature called "Blur" for videos, which is considered more natural and realistic than the iPhone's Cinema Mode.
   - The Pixel 9 Pro also has a night vision mode for recordings in poor light, which is not mentioned for the iPhone models.

3. **Selfie Cameras:**
   - **iPhone 15 Pro Max (assuming similar to iPhone 16 Pro):** 12MP selfie camera with autofocus and Portrait mode.
   - **Google Pixel 9 Pro:** 42MP selfie camera (output at 10.5MP) with autofocus and Portrait mode. The Pixel's selfie camera offers more detail and nicer colors compared to the iPhone.

4. **Special Features:**
   - The iPhone 16 Pro offers superior color transitions when zooming in and out.
   - The Google Pixel 9 Pro has a better night vision mode for low-light recordings and a more natural blur effect for videos.
   - Both phones offer various camera modes like photography, video, and time-lapse.

5. **Low-light Performance:**
   - The iPhone 16 Pro is mentioned to have better night performance, but it's noted that this is only true if you don't use 8K recording.

In conclusion, both the iPhone 16 Pro and Google Pixel 9 Pro offer extraordinary camera functions with fine differences. The Pixel 9 Pro stands out due to its tele-zoom, great natural blur, and night vision mode, while the iPhone 16 Pro offers superior color transitions and better night performance (excluding 8K). The choice between the two may come down to personal preference for natural vs. enhanced effects and specific use cases. The iPhone 15 Pro's camera capabilities are assumed to be similar to the iPhone 16 Pro based on the provided information. 


# Task_4 Result: 
 Based on the provided information, here's a detailed comparison of the battery life and charging capabilities of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro:

### Battery Capacity:
- **Google Pixel 9 Pro**: 4,700 mAh
- **iPhone 16 Pro**: 3,582 mAh
- **iPhone 15 Pro**: 3,290 mAh

### Battery Life:
- **Google Pixel 9 Pro**:
  - Battery Life Estimate: 6 hours 53 minutes
  - Browsing: 17 hours
  - Video: 10 hours 11 minutes
  - In a separate test, it lasted 13 hours and 37 minutes surfing the web over cellular.

- **iPhone 16 Pro**:
  - Battery Life Estimate: 6 hours 35 minutes
  - Browsing: 16 hours 35 minutes
  - Video: 8 hours 29 minutes
  - It takes longer to drain compared to the iPhone 15 Pro, which lasted 10 hours and 53 minutes in a similar test.

- **iPhone 15 Pro**:
  - It fell short of 11 hours in battery tests, indicating it has the shortest battery life among the three devices.

### Charging Capabilities:
- **Google Pixel 9 Pro**:
  - Wired Charging: 37W, fully charges in 1 hour 21 minutes
  - Wireless Charging: 23W (Pixel Stand), 12W (Qi)
  - Reverse Wireless Charging: 5W
  - 30-minute charge: 53% (wired), 20% (wireless)

- **iPhone 16 Pro**:
  - Wired Charging: 38W, fully charges in 1 hour 34 minutes
  - Wireless Charging: 25W (MagSafe), 15W (Qi)
  - Reverse Wireless Charging: 4.5W
  - 30-minute charge: 58% (wired), 50% (wireless)

- **iPhone 15 Pro**:
  - No specific charging details were provided, but it's reasonable to assume it has similar charging capabilities to the iPhone 16 Pro, given the incremental updates between iPhone models.

### Summary:
- The **Google Pixel 9 Pro** has the largest battery capacity and generally offers the best battery life among the three devices. It also supports fast wired and wireless charging, although its charging times are slightly slower than the iPhone 16 Pro.
- The **iPhone 16 Pro** has a smaller battery capacity than the Pixel 9 Pro but still offers respectable battery life. It supports slightly faster wired and wireless charging compared to the Pixel 9 Pro and charges up more in a 30-minute period.
- The **iPhone 15 Pro** has the smallest battery capacity and the shortest battery life among the three devices. Specific charging details were not provided, but it likely has similar charging capabilities to the iPhone 16 Pro.

In conclusion, if battery life is a top priority, the **Google Pixel 9 Pro** is the best choice due to its larger battery capacity and longer battery life. However, if faster charging is more important, the **iPhone 16 Pro** might be a better option due to its slightly faster charging speeds and higher 30-minute charge percentages. The **iPhone 15 Pro** falls short in battery life compared to the other two devices. 


# Task_5 Result: 
 Based on the provided information, here's a detailed price comparison and value for money assessment of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro:

1. **Pricing:**
   - iPhone 16 Pro: Starts at $1,099
   - iPhone 15 Pro: Starts at $999 (new), but can be found renewed on Amazon for around $600-$650 (128GB)
   - Google Pixel 9 Pro: Starts at $799

2. **Value for Money:**
   - **iPhone 16 Pro:** The iPhone 16 Pro comes with a higher price tag, offering some improvements over the iPhone 15 Pro. However, the upgrades are minor, and the iPhone 15 Pro is still considered an incredible phone with a fantastic user experience and a powerful camera system. If you're looking for the latest features and don't mind paying a premium, the iPhone 16 Pro could be worth it. But given the minor upgrades, you might find better value in the iPhone 15 Pro, especially if you can find it renewed at a significant discount.

   - **iPhone 15 Pro:** The iPhone 15 Pro offers better specs compared to the iPhone 16 Pro in some aspects. With the release of the iPhone 16 Pro, you can find great deals on the iPhone 15 Pro, making it an attractive option. A renewed iPhone 15 Pro on Amazon can be found for around $600-$650 (128GB), which is significantly less than the starting price of the iPhone 16 Pro. This makes the iPhone 15 Pro an excellent value for money, especially if you don't need the latest features.

   - **Google Pixel 9 Pro:** The Google Pixel 9 Pro is the most affordable option, starting at $799. It offers a comparable experience to the iPhone 16 Pro in many aspects, except for raw performance. The Pixel 9 Pro is known for its excellent camera performance, which can rival many dedicated cameras. Additionally, the Pixel 8 Pro, which costs just $700, delivers a similar experience to the Pixel 9 Pro XL and comes with a 7-year update policy. This makes the Pixel 8 Pro an even more affordable option with great value for money.

3. **Recommendation:**
   - If you're looking for the latest features and don't mind paying a premium, the iPhone 16 Pro could be worth it.
   - If you're looking for the best value for money, the iPhone 15 Pro (especially renewed) or the Google Pixel 9 Pro (or even the Pixel 8 Pro) would be excellent choices. The iPhone 15 Pro offers better specs compared to the iPhone 16 Pro in some aspects and can be found at a significant discount. The Google Pixel 9 Pro offers a comparable experience to the iPhone 16 Pro in many aspects at a lower price point, and the Pixel 8 Pro offers even better value for money.

In conclusion, the iPhone 15 Pro and Google Pixel 9 Pro (or Pixel 8 Pro) offer the best value for money compared to the iPhone 16 Pro. The choice between these options depends on your specific needs and preferences, such as camera performance, software updates, and overall user experience. 


In [16]:
print("\n=== FINAL SYNTHESIZED RESPONSE ===")

display(Markdown(result["final_response"]))


=== FINAL SYNTHESIZED RESPONSE ===


Based on the detailed analyses provided by the specialized agents, here's a comprehensive comparison of the iPhone 16 Pro, iPhone 15 Pro, and Google Pixel 9 Pro to help you make an informed decision:

**Design:**

* The iPhone 15 Pro stands out with its premium titanium and glass build, offering a more luxurious feel and improved durability. It also features a higher refresh rate display for smoother scrolling and better responsiveness.
* The iPhone 16 Pro has an aluminum and glass design with a smaller camera module, but it lacks the premium feel and higher refresh rate of the iPhone 15 Pro.
* The Google Pixel 9 Pro has a cleaner design with USB-C ports and side buttons, similar in size and feel to the iPhone 16 Pro but may not match the premium build quality of the iPhone 15 Pro.

**Performance:**

* The iPhone 16 Pro is expected to outperform both the iPhone 15 Pro and Google Pixel 9 Pro with its more powerful A18 Pro chip and better graphics capabilities.
* The iPhone 15 Pro, powered by the A17 Pro chip, also outperforms the Pixel 9 Pro in benchmarks.
* While the Pixel 9 Pro has more RAM (16GB compared to 8GB in iPhones), the iPhone models offer better benchmark performance and battery life.

**Camera:**

* Both the iPhone 16 Pro and Google Pixel 9 Pro offer extraordinary camera functions with fine differences. The Pixel 9 Pro stands out due to its tele-zoom, great natural blur, and night vision mode, while the iPhone 16 Pro offers superior color transitions and better night performance (excluding 8K).
* The iPhone 16 Pro has a 48MP main camera, 12MP ultra-wide, and 12MP telephoto cameras. The Google Pixel 9 Pro has a 50MP main camera, but details about its ultra-wide and telephoto cameras were not provided.
* The Pixel 9 Pro has a 42MP selfie camera (output at 10.5MP) with more detail and nicer colors compared to the iPhone's 12MP selfie camera.

**Battery Life and Charging:**

* The Google Pixel 9 Pro has the largest battery capacity (4,700 mAh) and generally offers the best battery life among the three devices. It supports fast wired and wireless charging, although its charging times are slightly slower than the iPhone 16 Pro.
* The iPhone 16 Pro has a smaller battery capacity (3,582 mAh) but still offers respectable battery life. It supports slightly faster wired and wireless charging compared to the Pixel 9 Pro and charges up more in a 30-minute period.
* The iPhone 15 Pro has the smallest battery capacity (3,290 mAh) and the shortest battery life among the three devices.

**Price and Value for Money:**

* The iPhone 16 Pro starts at $1,099, offering some improvements over the iPhone 15 Pro. However, the upgrades are minor, and the iPhone 15 Pro is still considered an incredible phone with a fantastic user experience and a powerful camera system.
* The iPhone 15 Pro starts at $999 (new) but can be found renewed on Amazon for around $600-$650 (128GB), making it an attractive option with better value for money.
* The Google Pixel 9 Pro starts at $799, offering a comparable experience to the iPhone 16 Pro in many aspects at a lower price point. The Pixel 8 Pro, which costs just $700, delivers a similar experience to the Pixel 9 Pro XL and comes with a 7-year update policy, offering even better value for money.

**Recommendation:**

* If you're looking for the latest features and don't mind paying a premium, the iPhone 16 Pro could be worth it.
* If you're looking for the best value for money, the iPhone 15 Pro (especially renewed) or the Google Pixel 9 Pro (or even the Pixel 8 Pro) would be excellent choices. The iPhone 15 Pro offers better specs compared to the iPhone 16 Pro in some aspects and can be found at a significant discount. The Google Pixel 9 Pro offers a comparable experience to the iPhone 16 Pro in many aspects at a lower price point, and the Pixel 8 Pro offers even better value for money.

Ultimately, the choice between these devices will depend on your specific needs and preferences, such as camera performance, software updates, overall user experience, and budget. Consider what features are most important to you and choose the device that best fits your requirements.